Jupyter notebook för att göra kluster för Livsmedelsverkets databas

In [6]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [7]:
# Variable to toggle printouts
debug = True

Vi har konverterat en nedladdad Excel-fil till csv och sen till Sqlite 3. 

Den innehåller alla Livsmedelsverkets 59 näringsämnen. Här jobbar vi bara med kolhydrater, fett och protein.

Först läser vi in från databasen och lägger värdena i arrayen *result*

In [8]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

#Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
#print (columnObject)
columns = np.array(columnObject).tolist()

conn.close()

In [9]:
for column in columns:
    print (column[0],column[1])

0 Livsmedelsnamn
1 Livsmedelsnummer
2 Energi_kcal
3 Energi_kJ
4 Kolhydrater_g
5 Fett_g
6 Protein_g
7 Fibrer_g
8 Vatten_g
9 Alkohol_g
10 Aska_g
11 Monosackarider_g
12 Disackarider_g
13 Sackaros_g
14 Fullkorn_totalt_g
15 Sockerarter_g
16 Summa_mattade_fettsyror_g
17 Fettsyra_4_0_10_0_g
18 Fettsyra_12_0_g
19 Fettsyra_14_0_g
20 Fettsyra_16_0_g
21 Fettsyra_18_0_g
22 Fettsyra_20_0_g
23 Summa_enkelomattade_fettsyror_g
24 Fettsyra_16_1_g
25 Fettsyra_18_1_g
26 Summa_fleromattade_fettsyror_g
27 Fettsyra_18_2_g
28 Fettsyra_18_3_g
29 Fettsyra_20_4_g
30 EPA_Fettsyra_20_5_g
31 DPA_Fettsyra_22_5_g
32 DHA_Fettsyra_22_6_g
33 Kolesterol_mg
34 Retinol_mikrog
35 Vitamin_A_mikrog
36 beta_Karoten_mikrog
37 Vitamin_D_mikrog
38 Vitamin_E_mg
39 Vitamin_K_mikrog
40 Tiamin_mg
41 Riboflavin_mg
42 Vitamin_C_mg
43 Niacin_mg
44 Niacinekvivalenter_mg
45 Vitamin_B6_mg
46 Vitamin_B12_mikrog
47 Folat_mikrog
48 Fosfor_mg
49 Jod_mikrog
50 Jarn_mg
51 Kalcium_mg
52 Kalium_mg
53 Magnesium_mg
54 Natrium_mg
55 Salt_g
56 Selen_

In [133]:
# Välj vilka näringsvärden som ska läsas ur databasen. Gör en lista av dem i valueList
valueList = [1,4,5,6]
columnsToQuery=[]
for index, item in enumerate(valueList):
    columnsToQuery.append(columns[item][1])

# Gör en sträng av listan av kolumner
columnsToQuery = ', '.join(columnsToQuery)
if debug:
    print (columnsToQuery)

Livsmedelsnummer, Kolhydrater_g, Fett_g, Protein_g


In [134]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select '+columnsToQuery+' from livs limit 300')
for row in rows:
        result.append(row)

rows = np.array(result)
print (rows)

conn.close()

[['1' '0.0' '71.0' '7.0']
 ['2' '0.0' '85.0' '2.8']
 ['3' '0.0' '100.0' '0.0']
 ..., 
 ['353' '4.4' '0.1' '1.0']
 ['354' '4.1' '0.15' '1.56']
 ['355' '4.7' '0.3' '1.13']]


Livsmedelsnumret ska inte vara med i klustringen så vi gör två matriser. 

allFactors innehåller Livsmedelsnumret, och dataSet allt utom Livsmedelsnumret

In [135]:
allFactors = np.array(result)
dataSet = allFactors[:,1:]

För klustringsalgoritmerna behöver vi använda numpy och göra en np-array

För fem livsmedel, kolumner för kolhydrater, fett och protein

<pre>[[   0.    71.     7. ]
 [   0.    85.     2.8]
 [   0.   100.     0. ]
 [   0.   100.     0. ]
 [   0.5   80.     0.5]]</pre>



In [136]:
if debug:
    print (dataSet)

[['0.0' '71.0' '7.0']
 ['0.0' '85.0' '2.8']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.5' '80.0' '0.5']
 ['0.5' '60.0' '0.4']
 ['0.3' '70.0' '0.2']
 ['0.3' '60.0' '0.2']
 ['0.0' '60.0' '0.0']
 ['0.0' '82.0' '0.0']
 ['0.4' '80.0' '0.1']
 ['0.5' '80.0' '0.5']
 ['3.0' '38.0' '0.0']
 ['2.5' '35.0' '0.1']
 ['6.2' '40.0' '0.51']
 ['0.41' '87.81' '1.46']
 ['3.92' '40.21' '2.16']
 ['12.33' '42.21' '2.46']
 ['13.1' '56.0' '1.6']
 ['1.07' '78.81' '1.44']
 ['3.32' '47.67' '0.74']
 ['8.0' '40.0' '0.5']
 ['5.5' '40.0' '0.3']
 ['0.5' '82.0' '0.4']
 ['0.48' '81.8' '0.55']
 ['0.48' '81.8' '0.55']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['0.0' '100.0' '0.0']
 ['9.7' '25.0' '0.5']
 ['9.5' '59.9' '1.0']
 ['24.0' '40.0' '0.5']
 ['19.7' '25.0' '0.5']
 ['8.0' '43.1' '0.5']
 ['15.0' '26.6' '1.9']
 ['20.8' '14.5' '1.0']
 ['13.1' '0.0' '1.6']
 ['1.5' '80.0' '1.0']
 ['10.9' '32.0' '0

Funktion för att göra själva klustringen enligt k-means.

K-means förklaras relativt enkelt här: http://bigdata-madesimple.com/possibly-the-simplest-way-to-explain-k-means-algorithm/

In [137]:
def cluster(dataSet):

    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    assignedCluster = kmeans.labels_

    if debug:
        print ("Centroids \n", centroids)
        print
        print (assignedCluster)

    return assignedCluster

När man kör klustringsalgoritmen returneras en array av vilket kluster varje livsmedel tillhör. 

Man kan även få ut mittpunkterna i varje kluster genom variabeln *centroids*

Ett exempel: 

<pre>
centroids:
[[   0.    100.      0.  ]
 [   0.25   82.5     1.65]
 [   0.     71.      7.  ]]

assignedCluster:
[2 1 0 0 1]</pre>

In [138]:
numClusters = 5
assignedCluster = cluster(dataSet)

Centroids 
 [[  6.63178571  42.20535714   2.07928571]
 [  9.65612903   3.42754839   3.44387097]
 [ 55.12953846   5.23092308   8.83338462]
 [  0.3475      87.7175       0.7125    ]
 [  2.06107143  21.33357143  23.42142857]]
[3 3 3 3 3 0 3 0 0 3 3 3 0 0 0 3 0 0 0 3 0 0 0 3 3 3 3 3 3 3 3 3 3 3 0 0 0
 0 0 0 1 1 3 0 0 0 1 1 1 0 0 0 3 1 1 2 2 2 2 2 1 1 1 1 1 1 1 4 4 4 4 4 4 4
 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 4 0 4 4 0 1 1 4 4 1 1 1 2 2 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]


In [139]:
# @todo Att beskriva: Varför behövde vi göra assignedcluster till en np array?
# Misstänker att det hade att göra med new, 
assignedCluster = np.array([assignedCluster])
print(assignedCluster)


[[3 3 3 3 3 0 3 0 0 3 3 3 0 0 0 3 0 0 0 3 0 0 0 3 3 3 3 3 3 3 3 3 3 3 0 0 0
  0 0 0 1 1 3 0 0 0 1 1 1 0 0 0 3 1 1 2 2 2 2 2 1 1 1 1 1 1 1 4 4 4 4 4 4 4
  4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 4 0 4 4 0 1 1 4 4 1 1 1 2 2 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1]]


In [140]:
# @todo rename _new_ to something clearer
new=np.concatenate((allFactors[:,:1], assignedCluster.T), axis=1)
print (new)

[['1' '3']
 ['2' '3']
 ['3' '3']
 ['4' '3']
 ['5' '3']
 ['6' '0']
 ['7' '3']
 ['8' '0']
 ['9' '0']
 ['10' '3']
 ['12' '3']
 ['13' '3']
 ['17' '0']
 ['18' '0']
 ['20' '0']
 ['21' '3']
 ['22' '0']
 ['23' '0']
 ['24' '0']
 ['25' '3']
 ['26' '0']
 ['27' '0']
 ['28' '0']
 ['29' '3']
 ['30' '3']
 ['31' '3']
 ['33' '3']
 ['34' '3']
 ['35' '3']
 ['37' '3']
 ['38' '3']
 ['39' '3']
 ['40' '3']
 ['41' '3']
 ['42' '0']
 ['43' '0']
 ['44' '0']
 ['45' '0']
 ['46' '0']
 ['47' '0']
 ['48' '1']
 ['49' '1']
 ['50' '3']
 ['51' '0']
 ['52' '0']
 ['53' '0']
 ['55' '1']
 ['56' '1']
 ['57' '1']
 ['58' '0']
 ['59' '0']
 ['60' '0']
 ['62' '3']
 ['63' '1']
 ['64' '1']
 ['65' '2']
 ['66' '2']
 ['67' '2']
 ['68' '2']
 ['69' '2']
 ['70' '1']
 ['71' '1']
 ['72' '1']
 ['73' '1']
 ['74' '1']
 ['75' '1']
 ['76' '1']
 ['77' '4']
 ['78' '4']
 ['79' '4']
 ['80' '4']
 ['81' '4']
 ['82' '4']
 ['83' '4']
 ['84' '4']
 ['85' '4']
 ['86' '4']
 ['87' '1']
 ['88' '4']
 ['89' '4']
 ['90' '4']
 ['91' '4']
 ['92' '4']
 ['93' '4']
 

I princip kan det här göras med många fler näringsvärden där varje näringsvärde blir en "dimension". 

Eftersom vi vill visa klustringen i ett 3D-diagram har vi begränsat oss till tre dimensioner i exemplet.

## Visualisering med Plotly

I Plotly svarar varje näringsvärde mot en axel.

För att Plotly ska kunna plotta varje axel för sig transponerar vi data så vi får en rad för x-axeln, en för y-axeln och en för z.

Ett exempel:

<pre>
[[   0.     0.     0.     0.     0.5]
 [  71.    85.   100.   100.    80. ]
 [   7.     2.8    0.     0.     0.5]]
</pre>

In [141]:
dataToChart = dataSet.transpose()

In [142]:
if debug:
    print (dataToChart)

[['0.0' '0.0' '0.0' '0.0' '0.5' '0.5' '0.3' '0.3' '0.0' '0.0' '0.4' '0.5'
  '3.0' '2.5' '6.2' '0.41' '3.92' '12.33' '13.1' '1.07' '3.32' '8.0' '5.5'
  '0.5' '0.48' '0.48' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '9.7'
  '9.5' '24.0' '19.7' '8.0' '15.0' '20.8' '13.1' '1.5' '10.9' '2.9' '8.5'
  '9.17' '10.05' '2.5' '1.18' '0.45' '6.39' '2.2' '6.7' '6.6' '49.5' '39.0'
  '56.0' '46.0' '49.0' '2.1' '9.1' '0.9' '2.9' '3.1' '3.6' '2.9' '1.5'
  '0.8' '1.4' '1.2' '0.0' '1.5' '0.5' '0.3' '0.1' '1.5' '2.0' '2.0' '1.5'
  '1.5' '1.5' '2.1' '0.9' '0.8' '1.5' '1.4' '1.4' '0.5' '1.8' '2.5' '2.4'
  '2.5' '3.8' '3.5' '0.2' '8.2' '2.4' '3.4' '4.7' '15.91' '1.4' '7.2'
  '4.84' '10.0' '54.6' '51.98' '4.84' '5.03' '10.9' '4.7' '4.84' '15.0'
  '14.3' '4.8' '4.84' '4.98' '5.2' '5.5' '15.0' '14.0' '8.8' '14.6' '13.0'
  '16.8' '4.84' '4.8' '4.8' '5.12' '4.89' '10.38' '10.48' '10.49' '4.66'
  '64.4' '57.6' '68.9' '63.3' '62.8' '68.0' '71.1' '65.7' '63.9' '65.3'
  '63.5' '62.1' '45.3' '64.7' '61.7' '67.7' 

För att visualisera vilka kluster som ett livsmedel ligger så ger vi varje kluster olika färger. I Plotly svarar detta mot att varje kluster hamnar i ett separat "trace"

Därför behöver vi gå igenom vårt *dataToChart* och separera ut vilka livsmedel som hör till vilket kluster.

Då behöver vi en tvådimensionell array där storleken på första dimensionen är antal kluster och storleken på andra dimensionen är tre (eftersom vi visualiserar tre dimensioner)

In [143]:
# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(3)] for _ in range(numClusters)]

Det är lite krångligt att visualisera den här strukturen. Så här blir den med fem kluster (numClusters) och tre faktorer för klustringen (fett, protein, kolhydrater) 
<pre>
[
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []]
]
</pre>

(Om man väljer att klustra med fler än tre faktorer är det svårt att visualisera alla, utan man måste välja bort någon faktor)

In [144]:
jobbigArray = np.array(segmentedData)
print (jobbigArray)

[]


In [145]:
for num, cluster in enumerate(assignedCluster[0]):
    if debug:
        print (str(num) + ' ' + str(cluster))
    segmentedData[cluster][0].append(dataToChart[0][num])
    segmentedData[cluster][1].append(dataToChart[1][num])
    segmentedData[cluster][2].append(dataToChart[2][num])
    #segmentedData[cluster][3].append(dataToChart[3][num])
        
jobbigArray = np.array(segmentedData)
print (jobbigArray.ndim)

0 3
1 3
2 3
3 3
4 3
5 0
6 3
7 0
8 0
9 3
10 3
11 3
12 0
13 0
14 0
15 3
16 0
17 0
18 0
19 3
20 0
21 0
22 0
23 3
24 3
25 3
26 3
27 3
28 3
29 3
30 3
31 3
32 3
33 3
34 0
35 0
36 0
37 0
38 0
39 0
40 1
41 1
42 3
43 0
44 0
45 0
46 1
47 1
48 1
49 0
50 0
51 0
52 3
53 1
54 1
55 2
56 2
57 2
58 2
59 2
60 1
61 1
62 1
63 1
64 1
65 1
66 1
67 4
68 4
69 4
70 4
71 4
72 4
73 4
74 4
75 4
76 4
77 1
78 4
79 4
80 4
81 4
82 4
83 4
84 4
85 4
86 4
87 4
88 4
89 4
90 4
91 0
92 0
93 4
94 0
95 4
96 4
97 0
98 1
99 1
100 4
101 4
102 1
103 1
104 1
105 2
106 2
107 1
108 1
109 1
110 1
111 1
112 1
113 1
114 1
115 1
116 1
117 1
118 1
119 1
120 1
121 1
122 1
123 1
124 1
125 1
126 1
127 1
128 1
129 1
130 1
131 1
132 1
133 1
134 2
135 2
136 2
137 2
138 2
139 2
140 2
141 2
142 2
143 2
144 2
145 2
146 2
147 2
148 2
149 2
150 2
151 2
152 2
153 2
154 2
155 2
156 2
157 2
158 2
159 2
160 2
161 2
162 2
163 2
164 2
165 2
166 2
167 2
168 2
169 2
170 2
171 2
172 2
173 2
174 2
175 2
176 2
177 2
178 2
179 2
180 2
181 2
182 2
183 2
184 2


In [146]:
if debug:
    print(segmentedData)

[[['0.5', '0.3', '0.0', '3.0', '2.5', '6.2', '3.92', '12.33', '13.1', '3.32', '8.0', '5.5', '9.7', '9.5', '24.0', '19.7', '8.0', '15.0', '10.9', '2.9', '8.5', '1.18', '0.45', '6.39', '2.4', '2.5', '3.5', '2.4'], ['60.0', '60.0', '60.0', '38.0', '35.0', '40.0', '40.21', '42.21', '56.0', '47.67', '40.0', '40.0', '25.0', '59.9', '40.0', '25.0', '43.1', '26.6', '32.0', '48.0', '32.5', '54.44', '61.05', '36.73', '33.0', '43.0', '35.0', '27.34'], ['0.4', '0.2', '0.0', '0.0', '0.1', '0.51', '2.16', '2.46', '1.6', '0.74', '0.5', '0.3', '0.5', '1.0', '0.5', '0.5', '0.5', '1.9', '0.8', '5.2', '1.2', '2.55', '2.77', '1.97', '10.2', '9.6', '4.0', '6.06']], [['20.8', '13.1', '9.17', '10.05', '2.5', '6.7', '6.6', '2.1', '9.1', '0.9', '2.9', '3.1', '3.6', '2.9', '2.0', '3.4', '4.7', '7.2', '4.84', '10.0', '4.84', '5.03', '10.9', '4.7', '4.84', '15.0', '14.3', '4.8', '4.84', '4.98', '5.2', '5.5', '15.0', '14.0', '8.8', '14.6', '13.0', '16.8', '4.84', '4.8', '4.8', '5.12', '4.89', '10.38', '10.48', '10

In [147]:
# create traces for plotly
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][2],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1


In [153]:
layout = go.Layout(
        scene=go.Scene(
            # @todo just nu är vilket ställe i valuelist som visualiseras hårdkodat här 
            # frågan är hur bäst lösa
            xaxis=go.XAxis(title=valueList[1], tickprefix=valueList[1], showtickprefix='first'),
            yaxis=go.YAxis(title=valueList[2], tickprefix=valueList[2], showtickprefix='first'),
            zaxis=go.ZAxis(title=valueList[3], tickprefix=valueList[3], showtickprefix='first')
        ),
        height = 800,
        width = 800,
    )

In [154]:
fig = go.Figure(data=traces, layout=layout)    

In [150]:
py.iplot(fig, filename='table1')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~superhamster/0 or inside your plot.ly account where it is named 'table1'


In [23]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

        #Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Kluster'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    conn.execute('ALTER TABLE livs ADD COLUMN Kluster REAL;')

#
conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

## Lägga in kluster i databasen


In [24]:
print(new)

for row in new:
    print (row[0])



[[ 656.3    4. ]
 [ 763.     2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 711.5    4. ]
 [ 534.2    4. ]
 [ 621.     4. ]
 [ 532.6    4. ]
 [ 530.6    4. ]
 [ 721.1    4. ]
 [ 709.5    4. ]
 [ 711.5    4. ]
 [ 348.2    0. ]
 [ 320.1    0. ]
 [ 381.     0. ]
 [ 784.3    2. ]
 [ 383.9    0. ]
 [ 435.3    0. ]
 [ 554.9    4. ]
 [ 707.7    4. ]
 [ 438.4    0. ]
 [ 388.3    0. ]
 [ 377.3    0. ]
 [ 728.8    4. ]
 [ 727.5    4. ]
 [ 727.5    4. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 884.3    2. ]
 [ 262.5    3. ]
 [ 572.4    4. ]
 [ 453.6    0. ]
 [ 303.1    3. ]
 [ 415.7    0. ]
 [ 303.9    3. ]
 [ 216.8    3. ]
 [  59.7    1. ]
 [ 717.6    4. ]
 [ 332.6    0. ]
 [ 457.4    0. ]
 [ 326.8    0. ]
 [ 147.5    1. ]
 [  66.7    1. ]
 [ 112.3    1. ]
 [ 496.3    0. ]
 [ 552.4    4. ]
 [ 359.5    0. ]
 [ 586.2    4. ]
 [ 223.5    3. ]
 [ 231.9    3. ]
 [ 386.     0. ]
 [ 459.6    0. ]
 [ 343.7    0. ]
 [ 259.6    3.

Dags att läsa ner resultatet av klustringen till databasen

In [25]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()


for row in new:
    print(row[1], row[0])
    sql = 'UPDATE livs SET Kluster = '+row[1]+' WHERE Livsmedelsnummer = "'+row[0]+'";'
    print(sql)
    curs.execute(sql)
#curs.execute('UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";')
conn.commit()
conn.close()

4.0 656.3


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [26]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
   
KlusterObject = []
for row in curs.execute('SELECT Livsmedelsnamn from livs WHERE Kluster = 0;'):
    #print(row)
    KlusterObject.append(row)

        
Kluster = np.array(KlusterObject)
print (Kluster)

conn.close()

[['Dressing majonnäs lätt fett ca 15%']
 ['Dressing fett 0%']
 ['Mimosasallad']
 ['Rödbetssallad gräddfil']
 ['Skaldjurssallad m majonnäsdressing']
 ['Bearnaisesås fryst el pulver tillagad']
 ['Hollandaisesås fryst el pulver tillagad']
 ['Keso färskost fett 4%']
 ['Keso färskost ananas apelsin fett 3%']
 ['Keso färskost chili pesto lök feta fett 3,5-5%']
 ['Färskost på filmjölk fett ca 8%']
 ['Färskost på filmjölk o gräddfil fett ca 14%']
 ['Kvarg färskost fett 1% Kesella lätt']
 ['Kvarg färskost fett 10% Kesella']
 ['Mjukost smältost fett 4%']
 ['Färskost lätt fett 15% typ Philadelphia']
 ['Färskost extra light fett 5% typ Philadelphia']
 ['Bröstmjölk humanmjölk fett 4,6%']
 ['Filmjölk fett 3%']
 ['Kondenserad mjölk konserv fett ca 8%']
 ['Långfil fett 3%']
 ['Lättfil fett 0,5%']
 ['Milkshake choklad jordgubb']
 ['Mjölk fett 3%']
 ['Yoghurt naturell fett 3 %']
 ['Fruktyoghurt fett 2,5%']
 ['Fruktyoghurt lätt fett 0,5%']
 ['Mjölk fett 4,2 % typ lantmjölk']
 ['A-fil  fett 3%']
 ['Mellan